In [163]:
import importlib

import numpy as np
import torch
from tqdm import tqdm

import train_classifier

In [198]:
importlib.reload(train_classifier)

<module 'train_classifier' from '/home/aplesner/code/Ensemble-everything-everywhere-recreating-results/train_classifier.py'>

In [199]:
net = train_classifier.get_network()

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 2.81 MiB is free. Including non-PyTorch memory, this process has 23.65 GiB memory in use. Of the allocated memory 22.71 GiB is allocated by PyTorch, and 625.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [192]:
trainloader, testloader = train_classifier.get_data_loaders()

Files already downloaded and verified
Files already downloaded and verified


In [193]:
criterion, optimizer = train_classifier.get_criterion_and_optimizer(net)

In [197]:
train_classifier.train_classifier(net, trainloader, criterion, optimizer)

Epoch 1 of 2 started


  0%|                                                                                                                                  | 0/391 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 2.81 MiB is free. Including non-PyTorch memory, this process has 23.65 GiB memory in use. Of the allocated memory 22.71 GiB is allocated by PyTorch, and 626.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [173]:
def eval_classifier(net, testloader):
    correct = torch.zeros(53, device=train_classifier.device)
    total = 0
    with torch.no_grad(), torch.autocast(device_type=train_classifier.device.type, dtype=torch.float16):
        net.eval()
        for images, labels in tqdm(testloader):
            
            images = images.to(train_classifier.device, non_blocking=True)
            labels = labels.to(train_classifier.device, non_blocking=True)

            labels = labels.unsqueeze(dim=-1)
            
            outputs = net(images)
            _, predicted = torch.max(outputs.data, dim=-1)
            total += labels.size(0)
            correct += (predicted == labels).sum(dim=0)
    return {
        "Accuracies": correct.cpu().numpy() / total
    }


In [174]:
eval_classifier(net, testloader)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.24it/s]


{'Accuracies': array([0.2287, 0.1455, 0.1747, 0.117 , 0.1647, 0.2001, 0.1644, 0.1666,
        0.1047, 0.1811, 0.1007, 0.1   , 0.1249, 0.1448, 0.164 , 0.1543,
        0.1797, 0.1081, 0.1896, 0.231 , 0.1033, 0.1385, 0.1539, 0.1001,
        0.1005, 0.1   , 0.2239, 0.1772, 0.2391, 0.2   , 0.1626, 0.1   ,
        0.1063, 0.1485, 0.127 , 0.1042, 0.1214, 0.2407, 0.2461, 0.2868,
        0.107 , 0.15  , 0.1326, 0.1488, 0.292 , 0.2147, 0.2503, 0.2687,
        0.1568, 0.195 , 0.1   , 0.1972, 0.1756], dtype=float32)}

In [60]:
128*2*4*(200704*(36+4+2)+401408*8+802816*3)/(1024**3)

13.3984375

In [81]:
# for i in range(len(net.fc_layers)):
#     print(f'Accuracy of {i}-th layer: {100 * correct[i] / total}')

Time1: 0.0001704692840576172
Time2: 0.0004665851593017578
Time3: 14.386648416519165
Time4: 0.003313779830932617
Time5: 0.07365989685058594
Batch time: 14.464377641677856
